In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import polars as pl
from absl_extra import logging_utils
from temporal_fusion_transformer.src.datasets.favorita import read_raw_csv, make_temporal_dataframe, downcast_dataframe

logging_utils.setup_logging()

In [ ]:
df = make_temporal_dataframe("../data/favorita")
df

2023-08-27 16:58:36,277:[favorita.py:200->make_temporal_dataframe()]:DEBUG: Adding trajectory identifier
2023-08-27 16:58:36,278:[favorita.py:207->make_temporal_dataframe()]:DEBUG: Removing returns data


Resampling to regular grid:   0%|          | 0/143658 [00:00<?, ?it/s]

In [ ]:
store_info = pl.read_csv("../data/favorita/stores.csv")
items = pl.read_csv("../data/favorita/items.csv")
transactions = pl.read_csv("../data/favorita/transactions.csv", try_parse_dates=True)
oil = (
    pl.read_csv("../data/favorita/oil.csv", try_parse_dates=True)
    .with_columns(pl.col("dcoilwtico").forward_fill())
    .filter(pl.col("dcoilwtico").is_not_null())
)

holidays = pl.read_csv("../data/favorita/holidays_events.csv", try_parse_dates=True)

In [ ]:
national_holidays = holidays.filter(pl.col("locale") == "National")
regional_holidays = holidays.filter(pl.col("locale") == "Regional").rename({"locale_name": "state"})
local_holidays = holidays.filter(pl.col("locale") == "Local").rename({"locale_name": "city"})

In [ ]:
temporal = (
    df.join(oil, on="date", how="left", suffix="oil_")
    .join(store_info, on="store_nbr", how="left", suffix="store_info")
    .join(items, on="item_nbr", how="left", suffix="items_")
    .join(transactions, on=["date", "store_nbr"], suffix="transactions_")
    .join(national_holidays, on="date", suffix="national_holidays_")
    .join(regional_holidays, on=["date", "sate"], suffix="regional_holidays_")
    .join(local_holidays, on=["date", "city"], suffix="local_holidays_")
    .sort("traj_id", "date")
)